In [1]:
import numpy as np
import pandas as pd
import time

import requests                     #This is for scraping
from bs4 import BeautifulSoup       #This is for scraping
import webbrowser                   #Scraping web
import csv

In [2]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import io 
from selenium.webdriver.common.keys import Keys

In [3]:
#conda install -c anaconda xlrd
#Installs stuff for excel

In [4]:
#This is the database that I want to upgrade with the scraped content
df = pd.read_excel("data.xlsx",squeeze=True, encoding ="utf-8",dtype=str)
index = df.index
values = df.values

In [5]:
df.head()

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995


In [6]:
Commitee_Codebook_2014_2018 = {
    197 : "Törvényalkotási bizottság",
    198 : "Gazdasági bizottság",
    199 : "Költségvetési bizottság", 
    200 : "Kulturális bizottság",
    201 : "Népjóléti bizottság",
    202 : "Igazságügyi bizottság",
    203 : "Külügyi bizottság",
    204 : "Mezőgazdasági bizottság ", 
    205 : "Vállalkozásfejlesztési bizottság",
    206 : "Fenntartható fejlődés bizottsága",
    207 : "Európai ügyek bizottsága",
    208 : "Honvédelmi és rendészeti bizottság",
    209 : "Nemzeti összetartozás bizottsága",
    210 : "Nemzetbiztonsági bizottság",
    211 : "Mentelmi bizottság",
    212 : "Az Alkotmánybíróság tagjait jelölő eseti bizottság",
    213 : "Magyarországi nemzetiségek bizottsága",
}
Committee_list = list(Commitee_Codebook_2014_2018.values())

In [7]:
#selenium login start
#It seems like we need selenium, since we got a dynamic webpage... unlucky
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')
print(soup.prettify())

<html class="ltr yui3-js-enabled js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface generatedcontent video audio localstorage sessionstorage webworkers no-applicationcache svg inlinesvg smil svgclippaths webkit js chrome chrome85 chrome85-0 win secure no-high-contrast-mode" dir="ltr" lang="hu-HU" style="">
 <div class="" id="yui3-css-stamp" style="position: absolute !important; visibility: hidden !important">
 </div>
 <head>
  <title>
   Képviselők ciklusonkénti adatai - Országgyűlés
  </title>
  <meta content="initial-scale=1.0, width=device-width" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <link charset="utf-8" data-senna-track="permanent" href="/o/frontend-th

In [10]:
#Let's see, what kind of links does the page contain. The goal is to click on the links of the parliament members.
#First, let's make the list of all the urls. 
url_list = []
for url in soup.find_all('a'):
    url_list.append(url)

#Now, let's take a look on our data. 
for i in range (len(url_list)):
    print(i)
    print(url_list[i])


0
<a href="#main-content">Ugrás a tartalomhoz</a>
1
<a href="https://www.parlament.hu/web/guest/orszaggyules">
<span>ORSZÁGGYŰLÉS</span>
</a>
2
<a href="https://www.parlament.hu/web/guest/bevezetes-jogi-szabalyzas1" role="menuitem">
<span>Bevezetés, jogi szabályozás</span>
</a>
3
<a href="https://www.parlament.hu/web/guest/mukodes-feladatok1" role="menuitem">
<span>Működés, feladatok</span>
</a>
4
<a href="https://www.parlament.hu/web/guest/nemzetpolitika" role="menuitem">
<span>Nemzetpolitika</span>
</a>
5
<a href="https://www.parlament.hu/web/guest/europai-unios-tevekenyseg" role="menuitem">
<span>Európai uniós tevékenység</span>
</a>
6
<a href="https://www.parlament.hu/web/guest/interparlamentaris-unio1" role="menuitem">
<span>Az Interparlamentáris Unió</span>
</a>
7
<a href="https://www.parlament.hu/web/guest/az-orszaggyules-barati-tagozatai" role="menuitem">
<span>Az Országgyűlés baráti tagozatai</span>
</a>
8
<a href="https://www.parlament.hu/web/orszaghaz">
<span>Országház</span

In [11]:
#At Access Date: 19/09/2020, the parliamentary members from A to Z are Ander Balázs to Zsigó Róbert. 
#Ander Balázs has a link at 231 number, Zsigó has at 464, so I will only keep these elements in the url_list. 
#It's important, that this might change in the future, but right now, my goal is to find and click on the links with the names. 
url_list = url_list[231:464]
print(url_list)

[<a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=NGYX2F5Q&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D">Ander Balázs</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=NGYX2F5Q&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da018%26p_ckl%3D40%26p_osszefuz%3D">Dr. Apáti István</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9

In [12]:
#I convert the url to string
for i in range (len(url_list)):
    url_list[i] = str(url_list[i])

In [13]:
#The href links won't direct me to the parliament member's page with request, I have to click on it. 
#All of the href elements has a name tag. I will collect all the names of the parliament members from the page. 
Names = []
for i in range(len(url_list)):
    if len(url_list[i]) > 400:
        name = (url_list[i][405:(len(url_list[i])-4)])
        Names.append(name)
#So, what do we have? 
print(Names)

['Ander Balázs', 'Dr. Apáti István', 'Dr. Aradszki András', 'Ágh Péter', 'B. Nagy László', 'Bajnai Gordon', 'Balczó Zoltán', 'Balla György', 'Balla Mihály', 'Balog Zoltán', 'Bana Tibor', 'Bangóné Borbély Ildikó', 'Bánki Erik', 'Bányai Gábor', 'Dr. Bárándy Gergely', 'Bartos Mónika', 'Becsó Zsolt', 'Bencsik János', 'Dr. Bene Ildikó', 'Bíró Márk', 'Bodó Sándor', 'Boldog István', 'Bóna Zoltán', 'Dr. Botka László', 'Budai Gyula', 'Burány Sándor', 'Csenger-Zalán Zsolt', 'Cseresnyés Péter', 'Csizi Péter', 'Csöbör Katalin', 'Czerván György', 'Dr. Czomba Sándor', 'Czunyiné Dr. Bertalan Judit', 'Dankó Béla', 'Demeter Márta', 'Demeter Zoltán', 'Dunai Mónika', 'Dúró Dóra', 'Egyed Zsolt', 'Farkas Flórián', 'Farkas Gergely', 'Farkas Sándor', 'Dr. Fazekas Sándor', 'Firtl Mátyás', 'Fodor Gábor', 'Dr. Fónagy János', 'Font Sándor', 'Földi László', 'Gaal Gergely', 'Dr. Galambos Dénes', 'Gelencsér Attila', 'Gőgös Zoltán', 'Gulyás Gergely', 'Gúr Nándor', 'Gyopáros Alpár', 'Gyöngyösi Márton', 'Győrffy Baláz

In [74]:
#Gives the party membership position of the person between 2014 and 2018
def get_party(soup):
    trigger = 0
    for element in soup.find_all('td'):
        element = str(element)
        if trigger == 1:
            result = element[4:-5]
            trigger = 2
        if element == "<td>2014-2018</td>" and trigger == 0:
            trigger = 1
    return result

def get_gov_support(party):
#I am decoding Party membership to a dummy variable, where 1 relates to government parties, 0 relates to opposition parties
    result = 0
    if party == "Fidesz" or party == "KDNP":
        result = 1
    return result

def get_exact_seat(soup):
    all_page_data = []
    result = 99999
    for element in soup.find_all('td'):
        element = str(element)
        all_page_data.append(element)
    
    for i in range (len(all_page_data)):
        if all_page_data[i] == '<td>2014.04.06.</td>':
            result = all_page_data[i+1][4:-9] 
    return(result)  
    

def get_committee_data(Committee_list, soup, toprint = False):
    
    all_page_data = []
    for element in soup.find_all('td'):
        element = str(element)
        all_page_data.append(element)
    if toprint == True:
        print(all_page_data)
    
    result = []
    for committee in Committee_list:
        for i in range (len(all_page_data)):

            if committee in all_page_data[i]:
                result.append(committee)
                result.append(all_page_data[i+1][4:-5])
                result.append(all_page_data[i+2][4:-5])
                result.append(all_page_data[i+3][4:-5])
    if result == []:
        result = 99995
    return result

def give_pandas_committee_data (name, committee_data, source_url, party, gov_support, exact_seat, seat_type):
    result = []
    if committee_data == 99995:
        new_row = {'name': name, 
                   'gov_support': gov_support, 
                   'source': source_url,
                   'committee_membership': 99995,
                   'cm_from_year': 99995,
                   'cm_from_month': 99995,
                   'cm_from_day': 99995,
                   'cm_to_year': 99995,
                   'cm_to_month': 99995,
                   'cm_to_day': 99995,
                   'seat_type': seat_type,
                   'exact_seat' : exact_seat,
                  }
        print(new_row)
        result.append(new_row)
        
    else:
        for i in range (len(committee_data)//4):
            new_row = {'name': name, 
                       'gov_support': gov_support, 
                       'source': source_url,
                       'committee_membership': committee_data[4*i],
                       'cm_from_year': committee_data[4*i+2][0:4],
                       'cm_from_month': committee_data[4*i+2][5:7],
                       'cm_from_day': committee_data[4*i+2][8:10],
                       'cm_to_year': committee_data[4*i+3][0:4],
                       'cm_to_month': committee_data[4*i+3][5:7],
                       'cm_to_day': committee_data[4*i+3][8:10],
                       'seat_type': seat_type,
                       'exact_seat' : exact_seat,
                      }
            print(new_row)
            result.append(new_row)
    return result

In [92]:
#It is time for the scrape loop, I click on the links with the Parliamentary members. 

all_rows = []
for i in range(len(Names)):
    driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
    #Next line: clicks on the element on the page, that contains the matching name. 
    link = driver.find_element_by_link_text(Names[i])
    link.click()
    source_url = driver.current_url
    html_source = driver.page_source #Makes soup
    soup = BeautifulSoup(html_source, 'html.parser')
    party = get_party(soup)
    exact_seat = get_exact_seat(soup)
    gov_support = get_gov_support(party)
    seat_type = 'SMD'
    if exact_seat == 'Országos l':
        seat_type = 'PL'
    if exact_seat == 'Országos l':
        seat_type = party
        
        
        
    committee_data = get_committee_data(Committee_list, soup)
    new_rows = give_pandas_committee_data(Names[i], committee_data, source_url, party, gov_support,exact_seat,seat_type)
    all_rows.append(new_rows)
    
    #print(Names[i])
    #print(driver.current_url)
    #print(party)
    #print(gov_support)
    #print(committee_data)
    
    #Sleep a bit, dude! 
    time.sleep(0.5)   

{'name': 'Ander Balázs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Kulturális bizottság', 'cm_from_year': '2018', 'cm_from_month': '06', 'cm_from_day': '11', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'Jobbik', 'exact_seat': 'Országos l'}
{'name': 'Ander Balázs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_

{'name': 'Ágh Péter', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da011%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Törvényalkotási bizottság', 'cm_from_year': '2020', 'cm_from_month': '05', 'cm_from_day': '05', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'SMD', 'exact_seat': 'Vas megye 2'}
{'name': 'Ágh Péter', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv

{'name': 'Balla György', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db007%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Törvényalkotási bizottság', 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '2018', 'cm_to_month': '10', 'cm_to_day': '15', 'seat_type': 'Fidesz', 'exact_seat': 'Országos l'}
{'name': 'Balla György', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcpls

{'name': 'Balog Zoltán', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db052%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 99995, 'cm_from_year': 99995, 'cm_from_month': 99995, 'cm_from_day': 99995, 'cm_to_year': 99995, 'cm_to_month': 99995, 'cm_to_day': 99995, 'seat_type': 'Fidesz', 'exact_seat': 'Országos l'}
{'name': 'Bana Tibor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat

{'name': 'Bányai Gábor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db053%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Vállalkozásfejlesztési bizottság', 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'SMD', 'exact_seat': 'Bács-Kiskun megye 5'}
{'name': 'Bányai Gábor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fintern

...

{'name': 'Varga Mihály', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv331%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Gazdasági bizottság', 'cm_from_year': '1998', 'cm_from_month': '06', 'cm_from_day': '25', 'cm_to_year': '1998', 'cm_to_month': '07', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Budapest 4'}
{'name': 'Vargha Tamás', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_

{'name': 'Vécsey László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv044%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Törvényalkotási bizottság', 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'SMD', 'exact_seat': 'Pest megye 6'}
{'name': 'Vécsey László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%

{'name': 'Vigh László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv038%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Igazságügyi bizottság', 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'SMD', 'exact_seat': 'Zala megye 1'}
{'name': 'Vigh László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kp

{'name': 'Vona Gábor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv058%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Külügyi bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'Jobbik', 'exact_seat': 'Országos l'}
{'name': 'Vona Gábor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv

{'name': 'Dr. Zombor Gábor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dz008%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Népjóléti bizottság', 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'SMD', 'exact_seat': 'Bács-Kiskun megye 2'}
{'name': 'Dr. Zombor Gábor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2F

In [93]:
import datetime

In [94]:
election_2014 = datetime.datetime(2014, 4, 6)
election_2018 = datetime.datetime(2018, 4, 8)
            
for rows in all_rows:
    for row in rows:
        values = list(row.values())
        if values [3] == 99995:
            df = df.append(row, ignore_index=True)
        else:
            start = (values[4]+'-'+values[5]+'-'+values[6])
            end   = (values[7]+'-'+values[8]+'-'+values[9])

            #Remove duplicate data
            if '20' in values[4] and '20' in values [7]:
                start = datetime.datetime.strptime(start, '%Y-%m-%d')
                end = datetime.datetime.strptime(end, '%Y-%m-%d')

                if election_2014 <= start <= election_2018:
                    print(row)
                    df = df.append(row, ignore_index=True)

{'name': 'Ander Balázs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Népjóléti bizottság', 'cm_from_year': '2016', 'cm_from_month': '09', 'cm_from_day': '12', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'Jobbik', 'exact_seat': 'Országos l'}
{'name': 'Ander Balázs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fo

{'name': 'Balla Mihály', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db008%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Külügyi bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Nógrád megye 2'}
{'name': 'Balla Mihály', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fog

{'name': 'Bartos Mónika', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db087%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Külügyi bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'Fidesz', 'exact_seat': 'Országos l'}
{'name': 'Bartos Mónika', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fo

{'name': 'Boldog István', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db091%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Költségvetési bizottság', 'cm_from_year': '2015', 'cm_from_month': '10', 'cm_from_day': '21', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Jász-Nagykun-Szolnok megye 4'}
{'name': 'Bóna Zoltán', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fin

{'name': 'Csöbör Katalin', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dc030%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Európai ügyek bizottsága', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Borsod-Abaúj-Zemplén megye 1'}
{'name': 'Dr. Czomba Sándor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairActi

...

{'name': 'Dr. Tapolczai Gergely', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt044%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Népjóléti bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'Fidesz', 'exact_seat': 'Országos l'}
{'name': 'Tasó László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcp

{'name': 'Tóth Csaba', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt045%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Gazdasági bizottság', 'cm_from_year': '2016', 'cm_from_month': '03', 'cm_from_day': '08', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Budapest 8'}
{'name': 'Törő Gábor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.

{'name': 'Vantara Gyula', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv043%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Gazdasági bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '02', 'seat_type': 'SMD', 'exact_seat': 'Békés megye 1'}
{'name': 'Vantara Gyula', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2

{'name': 'Dr. Vas Imre', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv054%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Honvédelmi és rendészeti bizottság', 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'SMD', 'exact_seat': 'Budapest 6'}
{'name': 'Vécsey László', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet

{'name': 'Z. Kárpát Dániel', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dz012%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 'Gazdasági bizottság', 'cm_from_year': '2016', 'cm_from_month': '03', 'cm_from_day': '08', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'Jobbik', 'exact_seat': 'Országos l'}
{'name': 'Z. Kárpát Dániel', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=w7yTvIn6&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcp

In [99]:
df

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995
703,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Ander Balázs,NaN,0,Országos l,Népjóléti bizottság,NaN,NaN,...,2016,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
704,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Ander Balázs,NaN,0,Országos l,Nemzeti összetartozás bizottsága,NaN,NaN,...,2014,12,09,2016,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Z. Kárpát Dániel,NaN,0,Országos l,Vállalkozásfejlesztési bizottság,NaN,NaN,...,2016,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
1063,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Dr. Zombor Gábor,NaN,1,Bács-Kiskun megye 2,Népjóléti bizottság,NaN,NaN,...,2014,23,06,2014,NaN,NaN,NaN,NaN,NaN,NaN
1064,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Dr. Zombor Gábor,NaN,1,Bács-Kiskun megye 2,Népjóléti bizottság,NaN,NaN,...,2015,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
1065,NaN,NaN,https://www.parlament.hu/web/guest/kepviselok-...,Zsigó Róbert,NaN,1,Bács-Kiskun megye 6,Külügyi bizottság,NaN,NaN,...,2014,07,06,2014,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
 df.drop_duplicates(keep = False, inplace = True) 

In [100]:
import openpyxl

In [101]:
df.to_excel("committee_timings_done.xlsx") 

In [90]:
#driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=NGYX2F5Q&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dz004%26p_ckl%3D40%26p_osszefuz%3D')
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')



u = get_committee_data(Committee_list,soup)
w = get_gov_support(soup)
x = get_party(soup)
v = get_exact_seat(soup)
seat_type = 'SMD'
if exact_seat == 'Országos l':
    seat_type = 'PL'
if exact_seat == 'Országos l':
    seat_type = x
y = seat_type

In [91]:
print(u)
print(w)
print(x)
print(v)
print(y)

99995
0
független
Országos l
SMD
